In [2]:
with open("the_verdict.txt","r", encoding = "utf-8") as f:
    raw_text = f.read()

print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20504
THE VERDICT
June 1908


I had always thought Jack Gisburn rather a cheap genius--though a
good fell


In [3]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [4]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [5]:
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [6]:
text = "Hello, world. Is this-- a test?"
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [7]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['THE', 'VERDICT', 'June', '1908', 'I', 'had', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to']


In [8]:
print(len(preprocessed))

4667


# Second step: Convert tolkens to tolken IDs

In [9]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1148


In [10]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [11]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
('1908', 8)
(':', 9)
(';', 10)
('?', 11)
('A', 12)
('AM', 13)
('Ah', 14)
('Among', 15)
('And', 16)
('Are', 17)
('Arrt', 18)
('As', 19)
('At', 20)
('Be', 21)
('Begin', 22)
('Burlington', 23)
('But', 24)
('By', 25)
('Carlo', 26)
('Chicago', 27)
('Claude', 28)
('Come', 29)
('Croft', 30)
('Destroyed', 31)
('Devonshire', 32)
('Don', 33)
('Dubarry', 34)
('Emperors', 35)
('End', 36)
('FELT', 37)
('Florence', 38)
('For', 39)
('Gallery', 40)
('Gideon', 41)
('Gisburn', 42)
('Gisburns', 43)
('Grafton', 44)
('Greek', 45)
('Grindle', 46)
('Grindles', 47)
('HAD', 48)
('HAS', 49)
('HAVE', 50)


In [12]:
class SimpleTokenizerV1:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text)
        return text

In [13]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know", 
Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

[1, 62, 2, 868, 1006, 619, 550, 764, 5, 1144, 614, 1, 5, 76, 7, 42, 869, 1126, 772, 812, 7]


In [14]:
text = tokenizer.decode(ids)
print(text)

" It' s the last he painted, you know", Mrs. Gisburn said with pardonable pride.


In [15]:
#text = "Hello, do you like tea?"
#print(tokenizer.encode(text))

In [16]:
all_tolkens = sorted(list(set(preprocessed)))
all_tolkens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tolkens)}

In [17]:
len(vocab.items())

1150

In [18]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1145)
('your', 1146)
('yourself', 1147)
('<|endoftext|>', 1148)
('<|unk|>', 1149)


In [19]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for 
                        item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])',r'\1', text)
        return text

In [20]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1,text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [21]:
tokenizer.encode(text)

[1149,
 5,
 372,
 1144,
 645,
 993,
 11,
 1148,
 61,
 1006,
 974,
 1002,
 740,
 1006,
 1149,
 7]

In [22]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

# Now we add Byte Pair Encoding

In [23]:
! pip3 install tiktoken

In [24]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.8.0


In [25]:
tokenizer = tiktoken.get_encoding("gpt2")

In [26]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"" of someunknownPlace")

integers = tokenizer.encode(text,allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 27271]


In [27]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPlace


In [28]:
integers = tokenizer.encode("Akwirw ier")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 86, 343, 86, 220, 959]
Akwirw ier


# Creating input-target data pairs

In [30]:
with open("the_verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5495


In [31]:
enc_sample = enc_text[50:]

In [33]:
context_size = 4 #Input size
# How many words to pay attention to at each time
# One input-output pair has context size amount of prediction tasks
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y: {y}")

x: [550, 5710, 465, 12036]
y: [5710, 465, 12036, 11]


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "-------->", desired)

[550] --------> 5710
[550, 5710] --------> 465
[550, 5710, 465] --------> 12036
[550, 5710, 465, 12036] --------> 11


In [38]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "----->",tokenizer.decode([desired]))

 had ----->  dropped
 had dropped ----->  his
 had dropped his ----->  painting
 had dropped his painting -----> ,


# Datasets and Dataloader

In [41]:
from torch.utils.data import Dataset,DataLoader
import torch 

class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids = []
        self.target_ids = []

        #Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        #Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]

In [42]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,
                         stride=128,shuffle=True,drop_last=True,
                         num_workers=0):
    
    #Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    #Create Dataset
    dataset = GPTDatasetV1(txt,tokenizer,max_length,stride)

    #Create dataloader
    dataloader = DataLoader(dataset,batch_size=batch_size,
                            shuffle=shuffle, drop_last=drop_last,
                            num_workers=num_workers)
    
    return dataloader

In [43]:
with open("the_verdict.txt","r",encoding="utf-8") as f:
    raw_text = f.read()

In [44]:
print("Pytorch version:", torch.__version__)
dataloader = create_dataloader_v1(raw_text,batch_size=1,max_length=4,
                                  stride=1,shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

Pytorch version: 2.4.1+cpu
[tensor([[10970, 33310,    35, 18379]]), tensor([[33310,    35, 18379,   198]])]


In [45]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[33310,    35, 18379,   198]]), tensor([[   35, 18379,   198, 15749]])]


In [46]:
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=4,
                                  stride=4,shuffle=False)

data_iter = iter(dataloader)
inputs, target = next(data_iter)
print("Inputs:\n", inputs)
print("Outputs:\n", target)

Inputs:
 tensor([[10970, 33310,    35, 18379],
        [  198, 15749, 40417,   628],
        [  198,    40,   550,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  198, 11274,  5891,  1576],
        [  438,   568,   340,   373]])
Outputs:
 tensor([[33310,    35, 18379,   198],
        [15749, 40417,   628,   198],
        [   40,   550,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   198],
        [11274,  5891,  1576,   438],
        [  568,   340,   373,   645]])
